In [912]:
import timeit
import time
import math
import numba as nb
import numpy as np
import numexpr as ne
import pandas as pd
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)

In [913]:
def py_damped_sine_func(x, A, lbda, omega, phi):
    return A * math.exp(-lbda * x) * math.sin(omega * x + phi)

def f_py_external(x_list, A, lbda, omega, phi, y_list):
    for i in range(len(x_list)):
        y_list[i] = py_damped_sine_func(x_list[i], A, lbda, omega, phi)

In [914]:
def f_numpy(x_array, A, lbda, omega, phi, y_array):
    return A * np.exp(-lbda * x_array) * np.sin(omega * x_array + phi)
    #np.sin(x_array, out=y_array)

In [915]:
@nb.jit(nopython=True, nogil=True, cache=True)
def numba_damped_sine_func(x, A, lbda, omega, phi):
    return A * np.exp(-lbda * x) * np.sin(omega * x + phi)              

@nb.jit(nopython=True, nogil=True, cache=True)
def f_numba(x_array, A, lbda, omega, phi, y_array):
    for i in range(x_array.size):
        y_array[i] = A * np.exp(-lbda * x_array[i]) * np.sin(omega * x_array[i] + phi)   
        
@nb.jit(nopython=True, nogil=True, cache=True)
def f_numba_external(x_array, A, lbda, omega, phi, y_array):
    for i in nb.prange(x_array.size):
        y_array[i] = numba_damped_sine_func(x_array[i], A, lbda, omega, phi)
        
@nb.jit(parallel=True, nopython=True, nogil=True, cache=True)
def f_numba_external_parall(x_array, A, lbda, omega, phi, y_array):
    for i in nb.prange(x_array.size):
        y_array[i] = numba_damped_sine_func(x_array[i], A, lbda, omega, phi)

In [916]:
def f_numexpr(x_array, A, lbda, omega, phi, y_array):
    ne.evaluate("A * exp(-lbda * x_array) * sin(omega * x_array + phi)", out=y_array)

In [917]:
funcs = [f_py_external, f_numpy, f_numba, f_numba_external_parall, f_numexpr]
sizes = [1, 1, 5, 10, 100, 500, 1_000, 2_000, 3_000, 5_000, 10_000, 20_000, 50_000, 100_000, 200_000, 500_000, 1_000_000]

In [918]:
funcs = [f_numpy, f_numba, f_numba_external_parall, f_numexpr]
sizes = [100, 200, 400, 600, 800, 1_000, 2_000, 3_000, 4_000, 5_000, 6_000, 7_000, 8_000, 9_000, 10_000, 12_000, 15_000]

In [919]:
times = np.zeros((len(sizes), len(funcs)))
compl_times = np.zeros(len(funcs))

In [920]:
A = 1.3
lbda = 1.7
omega = 8 * np.pi
phi = 0.5 * np.pi

In [921]:
REPEAT = 7
NUMBER_NOPY = 4_000
NUMBER_PY = 2
DTYPE = np.float64

In [922]:
# Compilation (1st run)
for j, f in enumerate(funcs):
    x_array = np.linspace(0, 1, num=1, dtype=DTYPE)
    y_array = np.empty_like(x_array)     
    if f.__name__.startswith('f_numba'):
        res = timeit.timeit(lambda: f(x_array, A, lbda, omega, phi, y_array), number=1)
    else:
        res = 0.0
    print(f'{f.__name__}: {res}')
    compl_times[j] = res

f_numpy: 0.0
f_numba: 0.005063636002887506
f_numba_external_parall: 0.005088444999273634
f_numexpr: 0.0


In [923]:
# Now, without compilation (multiple runs)    
for i, s in enumerate(sizes):
    for j, f in enumerate(funcs):
        x_array = np.linspace(0, 1, num=s, dtype=DTYPE)
        y_array = np.empty_like(x_array)     
        N = NUMBER_NOPY
        if f.__name__.startswith('f_py'):
            x_array = x_array.tolist()
            y_array = y_array.tolist()
            N = NUMBER_PY
        res = timeit.repeat(lambda: f(x_array, A, lbda, omega, phi, y_array), repeat=REPEAT, number=N)
        times[i, j] = np.sum(res) / REPEAT / NUMBER
        print(".", end="")
    print(s)

....100
....200
....400
....600
....800
....1000
....2000
....3000
....4000
....5000
....6000
....7000
....8000
....9000
....10000
....12000
....15000


In [924]:
l = [pd.DataFrame({'name': f.__name__, 
                   'x': sizes, 
                   'y': times[:, j],
                   'y_full': times[:, j] + compl_times[j]
                  }) 
     for j, f in enumerate(funcs)]
df = pd.concat(l)

In [930]:
fig = px.line(df, x='x', y='y', color='name', 
              log_x=True, log_y=True, markers=True, title='Without compilation time')
fig.update_xaxes(range=[1.95, 4.2])
fig.show()

In [932]:
fig = px.line(df, x='x', y='y_full', color='name', 
              log_x=True, log_y=True, markers=True, title='Including compilation time')
fig.update_xaxes(range=[1.95, 4.2])
fig.show()